## Trial-runs for model evaluation step

In [96]:
import os 

In [97]:
os.chdir(r"C:\Users\USER\Desktop\MLDefaults\Rising-Village-Prediction-Model")

In [98]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/samaTech-sys/Rising-Village-Prediction-Model.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "SamaTech-sys"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "27c279e0895284a35a6198b88076b93c13073cab"

## Trial-runs for entity_config file

In [99]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path 
    model_path: Path 
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

## Trial-runs for the ConfigarationManager

In [100]:
#importing all project paths and modules necessary for project configurations 
from raisingVillage.constants import *
from raisingVillage.utils.common import read_yaml, create_directories

In [101]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH, 
        params_filepath = PARAMS_FILE_PATH,
        selected_schema_filepath = SELECTED_SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.selected_schema = read_yaml(selected_schema_filepath)
       
        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation  # Should probably use model_evaluation section
        data_splitting_config = self.config.data_splitting
        model_training_config = self.config.model_training
        
        # Combine all parameters into one dictionary
        all_params = {
            "tfidf_params": self.params.model_training.TfidfVectorizer,
            "gb_params": self.params.model_training.GradientBoostingClassifier
        }
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=Path(config.root_dir),
            test_data_path=Path(data_splitting_config.test_set_path),
            model_path=Path(model_training_config.model_path),
            all_params=all_params,
            metric_file_name=Path(config.metric_file_name),
            target_column=self.selected_schema.TARGET,
            mlflow_uri="https://dagshub.com/samaTech-sys/Rising-Village-Prediction-Model.mlflow"
        )
        return model_evaluation_config

## Trial-runs for components 

In [102]:
import json
import joblib
import pandas as pd
import numpy as np
from raisingVillage import logger
import mlflow
from sklearn.metrics import (
                            accuracy_score, 
                            precision_score, 
                            recall_score,
                            f1_score, 
                            roc_auc_score, 
                            confusion_matrix,
                            classification_report)
from urllib.parse import urlparse

In [103]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
    def eval_metrics(self, actual, pred, proba=None):
        """Calculate all evaluation metrics with proper type handling"""
        # Convert predictions to binary if needed (threshold at 0.5)
        if np.issubdtype(pred.dtype, np.floating):
            pred = (pred >= 0.5).astype(int)
        
        # Ensure actual values are integers
        actual = actual.astype(int)
        
        metrics = {
            "accuracy": accuracy_score(actual, pred),
            "precision": precision_score(actual, pred, zero_division=0),
            "recall": recall_score(actual, pred, zero_division=0),
            "f1_score": f1_score(actual, pred, zero_division=0),
            "confusion_matrix": confusion_matrix(actual, pred).tolist(),
            "classification_report": classification_report(actual, pred, output_dict=True, zero_division=0)
        }
        
        if proba is not None:
            metrics["roc_auc"] = roc_auc_score(actual, proba)
        return metrics

    def log_into_mlflow(self):
        """Main method to execute full evaluation workflow"""
        try:
            # Load data and model
            test_data = pd.read_csv(str(self.config.test_data_path))
            model = joblib.load(str(self.config.model_path))
            
            # Handle target column
            target_column = "target_binary"
            if target_column not in test_data.columns:
                raise ValueError(f"Target column '{target_column}' not found in test data")
            
            # Prepare test data
            test_x = test_data.drop([target_column], axis=1)
            test_y = test_data[target_column].astype(int)  # Ensure binary target
            
            # Set MLflow tracking URI
            mlflow.set_registry_uri(self.config.mlflow_uri)
            
            with mlflow.start_run():
                # Get predictions and probabilities
                predictions = model.predict(test_x)
                probabilities = model.predict_proba(test_x)[:, 1] if hasattr(model, "predict_proba") else None
                
                # Calculate metrics
                metrics = self.eval_metrics(test_y, predictions, probabilities)
                
                # Save and log metrics
                self._save_metrics(metrics)
                params_to_log = dict(self.config.all_params) if hasattr(self.config.all_params, 'to_dict') else self.config.all_params
                mlflow.log_params(params_to_log)
                
                mlflow.log_metrics({
                    "accuracy": metrics["accuracy"],
                    "precision": metrics["precision"],
                    "recall": metrics["recall"],
                    "f1_score": metrics["f1_score"],
                })
                
                if "roc_auc" in metrics:
                    mlflow.log_metric("roc_auc", metrics["roc_auc"])
                
                # Model registry
                tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
                if tracking_url_type_store != "file":
                    mlflow.sklearn.log_model(model, "model", registered_model_name="GradientBoostingModel")
                else:
                    mlflow.sklearn.log_model(model, "model")
                    
        except Exception as e:
            logger.error(f"Error during model evaluation: {str(e)}")
            raise RuntimeError(f"Model evaluation failed: {str(e)}") from e
    
    def _save_metrics(self, metrics):
        """Save metrics to JSON file"""
        try:
            self.config.root_dir.mkdir(parents=True, exist_ok=True)
            metrics_path = self.config.root_dir / "metrics.json"
            with open(metrics_path, "w") as f:
                json.dump(metrics, f, indent=4)
                
            report_path = self.config.root_dir / "classification_report.json"
            with open(report_path, "w") as f:
                json.dump(metrics["classification_report"], f, indent=4)
        except Exception as e:
            logger.error(f"Failed to save metrics: {str(e)}")
            raise

## Trial-runs for pipeline

In [104]:
#Update pipeline Step 
try: 
    config = ConfigurationManager()
    model_evaluation_config =config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-05-29 19:09:15,286: INFO: common: yaml_file: config\config.yaml loaded successfully]
[2025-05-29 19:09:15,302: INFO: common: yaml_file: params.yaml loaded successfully]
[2025-05-29 19:09:15,302: INFO: common: yaml_file: selected_schema.yaml loaded successfully]
[2025-05-29 19:09:15,302: INFO: common: Created directory at: artifacts]
[2025-05-29 19:09:15,318: INFO: common: Created directory at: artifacts/model_evaluation]


Successfully registered model 'GradientBoostingModel'.


[2025-05-29 19:10:08,617: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /samaTech-sys/Rising-Village-Prediction-Model.mlflow/api/2.0/mlflow/runs/get?run_uuid=e793ca1936e042d2986ebb3afa6edcc5&run_id=e793ca1936e042d2986ebb3afa6edcc5]
[2025-05-29 19:10:34,512: WARNING: connectionpool: Retrying (Retry(total=3, connect=4, read=4, redirect=5, status=5)) after connection broken by 'ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001E7C7DDB940>, 'Connection to dagshub.com timed out. (connect timeout=120)')': /samaTech-sys/Rising-Village-Prediction-Model.mlflow/api/2.0/mlflow/runs/get?run_uuid=e793ca1936e042d2986ebb3afa6edcc5&run_id=e793ca1936e042d2986ebb3afa6edcc5]


2025/05/29 19:10:35 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: GradientBoostingModel, version 1
Created version '1' of model 'GradientBoostingModel'.
